# GitHub

## Git Commit and Push

In [ ]:
%cd "/content/drive/My Drive/Projects/QAMP Spring 2023/qamp-spring-23"

In [ ]:
# Check status
!git status

In [ ]:
# Add/track changed files

# only modified files
#!git add -u

# all files
!git add .

In [ ]:
# Commit the changes
# DON'T FORGET TO CHANGE THE COMMIT MESSAGE
safety_net = input('Have you changed the Commit Message? (Y/N)')
if safety_net == 'Y':
  !git commit -m "Update notebook." # commit message
  !git config --global user.email "eraraya.ricardo19@gmail.com"
  !git config --global user.name "ericardomuten"
else:
  print('Please change the Commit Message first.')

In [ ]:
# Push changes to GitHub Repo
!git push

In [ ]:
# Check status
!git status

## Git Pull

In [ ]:
%cd "/content/drive/My Drive/Projects/QAMP Spring 2023/qamp-spring-23"

In [ ]:
!git pull

In [ ]:
# Check status
!git status

# QAMP Spring 2023

## Resources, Examples, References

- https://github.com/Qiskit/qiskit-machine-learning/blob/main/qiskit_machine_learning/neural_networks/two_layer_qnn.py
- https://qiskit.org/documentation/stubs/qiskit.circuit.library.RealAmplitudes.html
- https://qiskit.org/ecosystem/machine-learning/tutorials/index.html
  - https://qiskit.org/ecosystem/machine-learning/tutorials/01_neural_networks.html
  - https://qiskit.org/ecosystem/machine-learning/tutorials/02a_training_a_quantum_model_on_a_real_dataset.html
  - https://qiskit.org/ecosystem/machine-learning/tutorials/05_torch_connector.html
- https://quantum-journal.org/papers/q-2020-02-06-226/pdf/ (original paper)

## List of Requirements (following the original paper)

### Ansatz
- Single qubit: \\
  Rotational Gate layers ($L$) only.

$$
\mathcal{U} = L(N)L(N-1) \ldots L(2)L(1) \\
\text{where:} \\
L(i) = U(\vec\theta_i^{(k)} + \vec w_i^{(k)} \cdot \vec x^{(k)} ) \ U(\vec\theta_i^{(k-1)} + \vec w_i^{(k-1)} \cdot \vec x^{(k-1)} ) \ldots \ U(\vec\theta_i^{(1)} + \vec w_i^{(1)} \cdot \vec x^{(1)} ) \\
\text{Each k vector is 3 dimmensional.} \\
U(\vec\theta) = Rx(\theta_x)Ry(\theta_y)Rz(\theta_z) \\
\vec\theta = (\theta_x, \theta_y, \theta_z) \\
\vec w = (w_x, w_y, w_z) \\
\vec x = \text{input data} \\
N = \text{number of layers}
$$

- Multi qubits without entanglement \\
  Stacking the same ansatz $\mathcal{U}$ for multiple qubits.

- Multi qubits with entanglement (CZ gates) \\
  Alternating between $L$ and entanglement layer $E$. \\
  **For 3 qubits or more? -> Linear entanglement**

$$
\mathcal{U} = L(N)\ E\ L(N-1)\ E \ldots E\ L(2)\ E\ L(1)
$$

### Classification & Measurement Strategy, Cost Function
Use the one-hot encoding (computational states).

- Single qubit ansatz
  - **Maximally orthogonal states (single qubit measurement)**:
      The correct states are the C maximally orthogonal states where C is the number of classes. Measure the fidelity between the output single qubit state vs the correct maximally orthogonal state. For example, for 4 classes, use the maximally orthogonal states formed by a tetrahedron inside a Bloch sphere.

$$
\chi^2(\vec{\alpha}, \vec{\theta}, \vec{w})=\frac{1}{2} \sum_{\mu=1}^M\left(\sum_{c=1}^{\mathcal{C}}\left(F_c\left(\vec{\theta}, \vec{w}, \vec{x}_\mu\right)-Y_c\left(\vec{x}_\mu\right)\right)^2\right)
$$

- Multi qubits ansatz
  - **Maximally orthogonal states (single qubit measurement)**: similar with the single qubit ansatz, but since we have multiple qubits now, we need to fix (choose) which one of the qubits we want to use for the measurement.

$$
\chi^2(\vec{\alpha}, \vec{\theta}, \vec{w})=\frac{1}{2} \sum_{\mu=1}^M\left(\sum_{c=1}^{\mathcal{C}}\left(F_c\left(\vec{\theta}, \vec{w}, \vec{x}_\mu\right)-Y_c\left(\vec{x}_\mu\right)\right)^2\right)
$$

  - **Maximally orthogonal states (average of multi single qubit measurement)**: similar like above, but instead of choosing 1 qubit, we do a single qubit measurement for all qubits and average the results.

$$
\chi^2(\vec{\alpha}, \vec{\theta}, \vec{w})= \frac{1}{2} \sum_{\mu=1}^M \sum_{c=1}^{\mathcal{C}}\left(\sum_{q=1}^Q\left(F_{c, q}\left(\vec{\theta}, \vec{w}, \vec{x}_\mu\right)-Y_c\left(\vec{x}_\mu\right)\right)^2\right)
$$

  - **Computational states (multi qubits measurement)**:
      The correct states are the computational states constructed from log2(C) qubits where C is the number of classes. Measure the fidelity between the output multi qubits state vs the correct computational state. For example, for 4 classes, the correct computational states for each classes are 00, 01, 10, and 11.

$$
\chi^2(\vec{\alpha}, \vec{\theta}, \vec{w})= \frac{1}{2} \sum_{\mu=1}^M \left(\sum_{q=1}^Q\left(F_{q}\left(\vec{\theta}, \vec{w}, \vec{x}_\mu\right)-Y_c\left(\vec{x}_\mu\right)\right)^2\right)
$$

### Fidelity Function $F$
$$
|\psi(\vec{\theta}, \vec{w}, \vec{x})\rangle = \mathcal{U} |0\rangle \text{, the output state.}
$$

- (Regular) Fidelity

$$
F_c(\vec{\theta}, \vec{w}, \vec{x})=\left|\left\langle\tilde{\psi}_c \mid \psi(\vec{\theta}, \vec{w}, \vec{x})\right\rangle\right|^2
$$

- Weighted Fidelity

$$
F_c(\vec{\theta}, \vec{w}, \vec{x})=\alpha_c\left|\left\langle\tilde{\psi}_c \mid \psi(\vec{\theta}, \vec{w}, \vec{x})\right\rangle\right|^2 \\
\text{The $\alpha_c$ is from $\vec \alpha = (\alpha_1, \alpha_2, \ldots, \alpha_C)$, which is another trainable parameters vector.}
$$

### Training/Optimization
How do we want to train the circuit? With TensorFlow? SciPy? PyTorch? \\
Integration with QNN implementation of Qiskit Machine Learning (EstimatorQNN and SamplerQNN)?

## Code Structure

In [ ]:
class DataReuploading():
    """
    Create the Data Reuploading Classifier ansatz.

    Notes: add QuantumInstance (backend) later.
    """

    def __init__(
        self,
        num_qubits: int | None = None,
        num_features: int | None = None,
        num_layers: int | None = None
    ):
        """
        Args:
            num_qubits: The number of qubits.
            num_features: The number of input features (the dimension of the input data).
            num_layers: The number of layers (N).
        Returns:
            ansatz: A QuantumCircuit object.
        """

    def rotational_gate_layer(
        self,
        num_qubits: int | None = None,
        num_features: int | None = None
    ):
        """
        Edo
          This is the L layer.
          Args:
              num_qubits: The number of qubits.
              num_features: The number of input features (the dimension of the input data).
          Returns:
              circuit_block: A QuantumCircuit object with ParameterVector as gate's arguments.
        """

    def entanglement_layer(
        self,
        num_qubits: int | None = None
    ):
        """
        Shivani
          This is the E layer.
          Args:
              num_qubits: The number of qubits.
          Returns:
              circuit_block: A QuantumCircuit object consists of only CZ gates.
        """

    @property
    def num_qubits(self) -> int:
        """Returns the number of qubits used by the ansatz."""
        return self._num_qubits

    @property
    def num_features(self) -> int:
        """Returns the number of features of the input data accepted by the ansatz."""
        return self._num_features

    @property
    def num_layers(self) -> int:
        """Returns the number of layers used by the ansatz."""
        return self._num_layers

- Example of training the ansatz with Iris Dataset.
  - Ansatz with single qubit
    - Binary classification (2 classes)
    - Multi class classification (3 classes)
  - Ansatz with multi qubits
    - Binary classification (2 classes)
    - Multi class classification (3 classes)